## **Run Model**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install latest pre-release version of bigdl-chronos 
# Installing bigdl-chronos from pip will automatically install pyspark, bigdl, and their dependencies.
#!pip install --pre --upgrade bigdl-chronos[all]
!pip install bigdl-chronos[all]==2.0.0
!pip install torchmetrics==0.7.3
!pip uninstall -y torchtext
#!pip install tensorflow-gpu==1.15.0
!pip install pmdarima
exit() # restart the runtime to refresh installed pkg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 222 kB 5.3 MB/s 
     |████████████████████████████████| 2.2 MB 59.9 MB/s 
     |████████████████████████████████| 9.9 MB 31.7 MB/s 
     |████████████████████████████████| 23.9 MB 1.4 MB/s 
     |████████████████████████████████| 71.0 MB 133 kB/s 
     |████████████████████████████████| 35.4 MB 556 kB/s 
     |████████████████████████████████| 101.0 MB 39 kB/s 
     |████████████████████████████████| 218.4 MB 62 kB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 57.6 MB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 70.7 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 271 kB 71.0 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 67.8 MB/s 
     |████████████████████████████████

## **Đánh giá theo độ dài dữ liệu**

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("/content/drive/MyDrive/Courses/Khoá luận tốt nghiệp/Dataset/final_data_6.csv")

df['datetime'] = df['datetime'].astype('datetime64')
df

#------------>>> IMPORT NECESARY LIBRARIES AND MODULES
from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca import OrcaContext
from bigdl.chronos.data import TSDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import bigdl.orca.automl.hp as hp
from bigdl.chronos.forecaster.arima_forecaster import ARIMAForecaster
from bigdl.orca.automl.metrics import Evaluator

#------------>>> RECOMMENDED TO SET IT TO TRUE WHEN RUNNING BIGDL-CHRONOS IN JUPYTER NOTEBOOK 
OrcaContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).

init_orca_context(cluster_mode="local", cores=4, init_ray_on_spark=True) # run in local mode



#------------>>> LOOP FOR FEATURES

tsdata_train, tsdata_val, tsdata_test = TSDataset.from_pandas(df, dt_col="datetime", id_col="id", target_col="value", with_split=True, val_ratio=0.3, test_ratio=0.1)

# minmax_scaler = MinMaxScaler()

# for tsdata in [tsdata_train, tsdata_val, tsdata_test]:
#   tsdata.scale(minmax_scaler, fit=(tsdata is tsdata_train))

# X_train, y_train = tsdata_train.to_numpy()
# X_val, y_val = tsdata_val.to_numpy()
# X_test, y_test = tsdata_test.to_numpy()
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

train_data = tsdata_train.to_pandas()
validation_data = tsdata_val.to_pandas()
test_data = tsdata_test.to_pandas()

print(train_data.shape[0])
print(validation_data.shape[0])

# BUILD MODEL
forecaster = ARIMAForecaster(p=2, q=2, seasonality_mode=True, P=3, Q=1, m=7)
forecaster.fit(train_data['value'].to_numpy(), test_data['value'].to_numpy())

# PREDICT
y_pred = forecaster.predict(horizon=19272, rolling=False)

# EVALUATE
forecaster.evaluate(test_data['value'].to_numpy(), metrics=['rmse', 'mae', 'mape', 'smape'])

/usr/local/lib/python3.7/dist-packages/bigdl/dllib/nncontext.py:377: UserWarning: Setting log_output takes no effect after the context has been initialized. You need to set log_output before initializing the context (e.g., before calling init_orca_context, init_nncontext, etc.)
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/bigdl/orca/common.py:233: Warning: Use an existing SparkContext, cluster_mode is determined by the existing SparkContext
  "cluster_mode is determined by the existing SparkContext", Warning)


Initializing orca context
2022-06-13 16:45:45,275 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 16:45:45,276 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 16:45:45,277 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 16:45:45,278 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-06-13 16:45:45 [Thread-3] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-06-13 16:45:45 [Thread-3] INFO  Engine$:123 - Executor number is 1 and executor cores number is 4
22-06-13 16:45:45 [Thread-3] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 16
2022-06-13 16:45:45 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-06-13 16:45:45 [Thread-3] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...
115632
57816


[array(1268.79036005), array(875.93777396), array(inf), array(1.19614285)]

In [19]:
# EVALUATE with Sklearn metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
def s_mape(a, f):
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f)))
    # return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

rmse = mean_squared_error(test_data['value'].to_numpy(), y_pred, squared=False)
mae = mean_absolute_error(test_data['value'].to_numpy(), y_pred)
mape = mean_absolute_percentage_error(test_data['value'].to_numpy(), y_pred)
smape = s_mape(test_data['value'].to_numpy(), y_pred)

print("Kết quả dự đoán trên tập test full:")
print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)
print('sMAPE:', smape)

Kết quả dự đoán trên tập test full:
RMSE: 1268.790360052456
MAE: 875.9377739602476
MAPE: 1.233695135166232e+16
sMAPE: 1.1961428493708377


### **Trừ 50 dòng dữ liệu mỗi sensor**

In [20]:
test_data['pred'] = y_pred
test_data

,id,datetime,value,irain,rain,itemp,temp,iwb,wetb,dewpt,...,GL3,1283,1258,1256,weekday,weekend,lat,long,holiday,pred
173448,000000001253A,2021-11-25 12:00:00,3110,0,0.0,0,5.4,0,4.0,2.0,...,0,0,0,0,3,0,51.87488,-8.44783,0,84.913517
173470,000000001253A,2021-11-25 13:00:00,3151,0,0.0,0,5.8,0,4.3,2.0,...,0,0,0,0,3,0,51.87488,-8.44783,0,91.437539
173492,000000001253A,2021-11-25 14:00:00,3067,0,0.0,0,6.1,0,4.3,1.6,...,0,0,0,0,3,0,51.87488,-8.44783,0,90.665556
173514,000000001253A,2021-11-25 15:00:00,3491,0,0.0,0,6.0,0,4.3,1.9,...,0,0,0,0,3,0,51.87488,-8.44783,0,91.864679
173536,000000001253A,2021-11-25 16:00:00,3594,0,0.0,0,5.5,0,3.9,1.5,...,0,0,0,0,3,0,51.87488,-8.44783,0,108.858040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192631,000000020223B,2021-12-31 19:00:00,210,0,0.4,0,11.6,0,11.6,11.5,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.945519
192653,000000020223B,2021-12-31 20:00:00,139,0,0.3,0,11.6,0,11.5,11.4,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.945253
192675,000000020223B,2021-12-31 21:00:00,69,2,0.0,0,11.4,0,11.2,11.1,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.944988
192697,000000020223B,2021-12-31 22:00:00,52,2,0.0,0,11.1,0,10.9,10.8,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.944722


In [21]:
list_id = ['000000001253A',
 '000000001253B',
 '000000001255A',
 '000000001255B',
 '000000001256A',
 '000000001256B',
 '000000001258A',
 '000000001258B',
 '000000001271A',
 '000000001271B',
 '000000001283A',
 '000000001283B',
 '000000001401A',
 '000000001401B',
 '000000001403A',
 '000000001403B',
 '000000001404A',
 '000000001404B',
 '000000001405A',
 '000000001405B',
 '000000020223A',
 '000000020223B']
result = pd.DataFrame()
for i in list_id:
  df_filter = test_data[test_data['id'] == i].iloc[50:,:]
  result = pd.concat([result, df_filter])

#result.to_csv("/content/drive/MyDrive/Courses/Khoá luận tốt nghiệp/Notebook/Notebook Model/Compare Model/Result_Forecaster_ARIMA_25gb.csv", index=False)
result

,id,datetime,value,irain,rain,itemp,temp,iwb,wetb,dewpt,...,GL3,1283,1258,1256,weekday,weekend,lat,long,holiday,pred
174548,000000001253A,2021-11-27 14:00:00,3395,0,0.0,0,5.1,0,2.7,-1.5,...,0,0,0,0,5,1,51.87488,-8.44783,0,159.109409
174570,000000001253A,2021-11-27 15:00:00,3174,0,0.0,0,4.7,0,2.5,-1.4,...,0,0,0,0,5,1,51.87488,-8.44783,0,159.315367
174592,000000001253A,2021-11-27 16:00:00,3312,0,0.0,0,4.2,0,2.3,-1.0,...,0,0,0,0,5,1,51.87488,-8.44783,0,159.617682
174614,000000001253A,2021-11-27 17:00:00,2785,0,0.0,0,3.5,0,1.9,-1.0,...,0,0,0,0,5,1,51.87488,-8.44783,0,159.849674
174636,000000001253A,2021-11-27 18:00:00,2225,3,0.0,0,3.0,0,1.5,-1.3,...,0,0,0,0,5,1,51.87488,-8.44783,0,160.131780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192631,000000020223B,2021-12-31 19:00:00,210,0,0.4,0,11.6,0,11.6,11.5,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.945519
192653,000000020223B,2021-12-31 20:00:00,139,0,0.3,0,11.6,0,11.5,11.4,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.945253
192675,000000020223B,2021-12-31 21:00:00,69,2,0.0,0,11.4,0,11.2,11.1,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.944988
192697,000000020223B,2021-12-31 22:00:00,52,2,0.0,0,11.1,0,10.9,10.8,...,0,0,0,0,4,0,51.88714,-8.56911,0,154.944722


In [22]:
# EVALUATE with Sklearn metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
def s_mape(a, f):
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f)))
    # return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

rmse = mean_squared_error(result['value'].to_numpy(), result['pred'].to_numpy(), squared=False)
mae = mean_absolute_error(result['value'].to_numpy(), result['pred'].to_numpy())
mape = mean_absolute_percentage_error(result['value'].to_numpy(), result['pred'].to_numpy())
smape = s_mape(result['value'].to_numpy(), result['pred'].to_numpy())

print("Kết quả dự đoán trên tập test đã cắt 50 dòng mỗi sensor:")
print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)
print('sMAPE:', smape)

Kết quả dự đoán trên tập test đã cắt 50 dòng mỗi sensor:
RMSE: 1258.1148206883609
MAE: 866.2568324460885
MAPE: 9166067724093608.0
sMAPE: 1.1879766962535034


## **Đánh giá theo tính cập nhật của mô hình**

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("/content/drive/MyDrive/Courses/Khoá luận tốt nghiệp/Dataset/final_data_6.csv")

df['datetime'] = df['datetime'].astype('datetime64')
df

#------------>>> IMPORT NECESARY LIBRARIES AND MODULES
from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca import OrcaContext
from bigdl.chronos.data import TSDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import bigdl.orca.automl.hp as hp
from bigdl.chronos.forecaster.arima_forecaster import ARIMAForecaster
from bigdl.orca.automl.metrics import Evaluator

#------------>>> RECOMMENDED TO SET IT TO TRUE WHEN RUNNING BIGDL-CHRONOS IN JUPYTER NOTEBOOK 
OrcaContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).

init_orca_context(cluster_mode="local", cores=4, init_ray_on_spark=True) # run in local mode



#------------>>> LOOP FOR FEATURES
_, tsdata_train, _ = TSDataset.from_pandas(df, dt_col="datetime", id_col="id", target_col="value", with_split=True, val_ratio=0.2, test_ratio=0.2)

tsdata_trainn, tsdata_val, tsdata_test = TSDataset.from_pandas(df, dt_col="datetime", id_col="id", target_col="value", with_split=True, val_ratio=0.7, test_ratio=0.1)

# minmax_scaler = MinMaxScaler()

# for tsdata in [tsdata_train, tsdata_val, tsdata_test]:
#   tsdata.scale(minmax_scaler, fit=(tsdata is tsdata_train))

# X_train, y_train = tsdata_train.to_numpy()
# X_val, y_val = tsdata_val.to_numpy()
# X_test, y_test = tsdata_test.to_numpy()
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

train_data = tsdata_train.to_pandas()
validation_data = tsdata_val.to_pandas()
test_data = tsdata_test.to_pandas()

print(train_data.shape[0])
print(validation_data.shape[0])

# BUILD MODEL
forecaster = ARIMAForecaster(p=2, q=2, seasonality_mode=True, P=3, Q=1, m=7)
forecaster.fit(train_data['value'].to_numpy(), test_data['value'].to_numpy())

# PREDICT
y_pred = forecaster.predict(horizon=19272, rolling=False)

# EVALUATE
forecaster.evaluate(test_data['value'].to_numpy(), metrics=['rmse', 'mae', 'mape', 'smape'])

/usr/local/lib/python3.7/dist-packages/bigdl/dllib/nncontext.py:377: UserWarning: Setting log_output takes no effect after the context has been initialized. You need to set log_output before initializing the context (e.g., before calling init_orca_context, init_nncontext, etc.)
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/bigdl/orca/common.py:233: Warning: Use an existing SparkContext, cluster_mode is determined by the existing SparkContext
  "cluster_mode is determined by the existing SparkContext", Warning)


Initializing orca context
2022-06-13 17:31:58,262 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 17:31:58,263 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 17:31:58,269 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 17:31:58,270 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-06-13 17:31:58 [Thread-3] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-06-13 17:31:58 [Thread-3] INFO  Engine$:123 - Executor number is 1 and executor cores number is 4
22-06-13 17:31:58 [Thread-3] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 16
2022-06-13 17:31:58 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-06-13 17:31:58 [Thread-3] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...
38544
134904


[array(1328.38754693), array(931.31307025), array(inf), array(1.42670056)]

In [36]:
# EVALUATE with Sklearn metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
def s_mape(a, f):
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f)))
    # return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

rmse = mean_squared_error(test_data['value'].to_numpy(), y_pred, squared=False)
mae = mean_absolute_error(test_data['value'].to_numpy(), y_pred)
mape = mean_absolute_percentage_error(test_data['value'].to_numpy(), y_pred)
smape = s_mape(test_data['value'].to_numpy(), y_pred)

print("Kết quả dự đoán trên tập test full:")
print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)
print('sMAPE:', smape)

Kết quả dự đoán trên tập test full:
RMSE: 1328.3875469266968
MAE: 931.3130702508935
MAPE: 6048004838138890.0
sMAPE: 1.4267005551661807


### **Trừ 50 dòng dữ liệu mỗi sensor**

In [37]:
test_data['pred'] = y_pred
test_data

,id,datetime,value,irain,rain,itemp,temp,iwb,wetb,dewpt,...,GL3,1283,1258,1256,weekday,weekend,lat,long,holiday,pred
173448,000000001253A,2021-11-25 12:00:00,3110,0,0.0,0,5.4,0,4.0,2.0,...,0,0,0,0,3,0,51.87488,-8.44783,0,4.292436
173470,000000001253A,2021-11-25 13:00:00,3151,0,0.0,0,5.8,0,4.3,2.0,...,0,0,0,0,3,0,51.87488,-8.44783,0,-3.370362
173492,000000001253A,2021-11-25 14:00:00,3067,0,0.0,0,6.1,0,4.3,1.6,...,0,0,0,0,3,0,51.87488,-8.44783,0,25.408339
173514,000000001253A,2021-11-25 15:00:00,3491,0,0.0,0,6.0,0,4.3,1.9,...,0,0,0,0,3,0,51.87488,-8.44783,0,52.360037
173536,000000001253A,2021-11-25 16:00:00,3594,0,0.0,0,5.5,0,3.9,1.5,...,0,0,0,0,3,0,51.87488,-8.44783,0,114.874204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192631,000000020223B,2021-12-31 19:00:00,210,0,0.4,0,11.6,0,11.6,11.5,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.101267
192653,000000020223B,2021-12-31 20:00:00,139,0,0.3,0,11.6,0,11.5,11.4,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.095356
192675,000000020223B,2021-12-31 21:00:00,69,2,0.0,0,11.4,0,11.2,11.1,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.089445
192697,000000020223B,2021-12-31 22:00:00,52,2,0.0,0,11.1,0,10.9,10.8,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.083534


In [38]:
list_id = ['000000001253A',
 '000000001253B',
 '000000001255A',
 '000000001255B',
 '000000001256A',
 '000000001256B',
 '000000001258A',
 '000000001258B',
 '000000001271A',
 '000000001271B',
 '000000001283A',
 '000000001283B',
 '000000001401A',
 '000000001401B',
 '000000001403A',
 '000000001403B',
 '000000001404A',
 '000000001404B',
 '000000001405A',
 '000000001405B',
 '000000020223A',
 '000000020223B']
result = pd.DataFrame()
for i in list_id:
  df_filter = test_data[test_data['id'] == i].iloc[50:,:]
  result = pd.concat([result, df_filter])

#result.to_csv("/content/drive/MyDrive/Courses/Khoá luận tốt nghiệp/Notebook/Notebook Model/Compare Model/Result_Forecaster_ARIMA_25gb.csv", index=False)
result

,id,datetime,value,irain,rain,itemp,temp,iwb,wetb,dewpt,...,GL3,1283,1258,1256,weekday,weekend,lat,long,holiday,pred
174548,000000001253A,2021-11-27 14:00:00,3395,0,0.0,0,5.1,0,2.7,-1.5,...,0,0,0,0,5,1,51.87488,-8.44783,0,119.529406
174570,000000001253A,2021-11-27 15:00:00,3174,0,0.0,0,4.7,0,2.5,-1.4,...,0,0,0,0,5,1,51.87488,-8.44783,0,120.404534
174592,000000001253A,2021-11-27 16:00:00,3312,0,0.0,0,4.2,0,2.3,-1.0,...,0,0,0,0,5,1,51.87488,-8.44783,0,121.003134
174614,000000001253A,2021-11-27 17:00:00,2785,0,0.0,0,3.5,0,1.9,-1.0,...,0,0,0,0,5,1,51.87488,-8.44783,0,121.764603
174636,000000001253A,2021-11-27 18:00:00,2225,3,0.0,0,3.0,0,1.5,-1.3,...,0,0,0,0,5,1,51.87488,-8.44783,0,122.018308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192631,000000020223B,2021-12-31 19:00:00,210,0,0.4,0,11.6,0,11.6,11.5,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.101267
192653,000000020223B,2021-12-31 20:00:00,139,0,0.3,0,11.6,0,11.5,11.4,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.095356
192675,000000020223B,2021-12-31 21:00:00,69,2,0.0,0,11.4,0,11.2,11.1,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.089445
192697,000000020223B,2021-12-31 22:00:00,52,2,0.0,0,11.1,0,10.9,10.8,...,0,0,0,0,4,0,51.88714,-8.56911,0,8.083534


In [39]:
# EVALUATE with Sklearn metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
def s_mape(a, f):
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f)))
    # return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

rmse = mean_squared_error(result['value'].to_numpy(), result['pred'].to_numpy(), squared=False)
mae = mean_absolute_error(result['value'].to_numpy(), result['pred'].to_numpy())
mape = mean_absolute_percentage_error(result['value'].to_numpy(), result['pred'].to_numpy())
smape = s_mape(result['value'].to_numpy(), result['pred'].to_numpy())

print("Kết quả dự đoán trên tập test đã cắt 50 dòng mỗi sensor:")
print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)
print('sMAPE:', smape)

Kết quả dự đoán trên tập test đã cắt 50 dòng mỗi sensor:
RMSE: 1317.6550284605103
MAE: 921.8567277749353
MAPE: 4485145315199588.5
sMAPE: 1.4198681970158755
